Zero shot classifier - https://huggingface.co/facebook/bart-large-mnli

In [1]:
import pandas as pd
from datasets import Dataset
from transformers import pipeline

In [2]:
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli") #built to understand natural language
candidate_labels = ['Hybrid', 'OnSite', 'Remote']

Device set to use cuda:0


Eval

In [3]:
#get test data

fp_test = "../DATASETS/work_arrangements_test_set.csv"
df_test = pd.read_csv(fp_test)
df_test.drop("id", axis=1, inplace=True) #get rid of id column
df_test.rename(columns={"job_ad": "text"}, inplace=True)

df_test
testdata = Dataset.from_pandas(df_test)


In [ ]:
correct = 0
predictions = []


for i in range(len(testdata)):
    sample = testdata[i]

    out = classifier(sample['text'], candidate_labels)
    scores = list(zip(out['labels'], out['scores']))

    predicted_label = max(scores, key=lambda x: x[1])[0]    #get label of largest confidence score
    predictions.append(predicted_label)
    if predicted_label == sample['y_true']:
        correct+=1

print(f"Accuracy = {correct/len(testdata)}")


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Accuracy = 0.494949494949495


In [ ]:
from eval import *

p = precision(["Remote", "Hybrid", "OnSite"], testdata['y_true'], predictions)
print(p)
r = recall(["Remote", "Hybrid", "OnSite"], testdata['y_true'], predictions)
print(r)

{'Remote': 0.8333333333333334, 'Hybrid': 0.34782608695652173, 'OnSite': 0.8333333333333334, 'AVERAGE': 0.6714975845410628}
{'Remote': 0.7692307692307693, 'Hybrid': 0.8888888888888888, 'OnSite': 0.10869565217391304, 'AVERAGE': 0.5889384367645237}
